In [1]:
import dgl_ptm
import os
os.environ["DGLBACKEND"] = "pytorch"

In [2]:
# Create an instance of the model class
model = dgl_ptm.PovertyTrapModel(model_identifier='test')

# Set the model and simulation parameters
# Using DEFAULT values
model.set_model_parameters(default=True)

# OR
# using a CONFIG file
# model.set_model_parameters(default=True, parameterFilePath='../dgl_ptm/dgl_ptm/config.yaml')

# Initialize the model
model.initialize_model()

In [6]:
model.step_target = 5

In [7]:
model.run()

performing step 1 of 5
creating bidirectional link between nodes 36 (src) and tensor([84]) (dst)
created 1 of 1 links requested
performing step 2 of 5
creating bidirectional link between nodes 24 (src) and tensor([1]) (dst)
created 1 of 1 links requested
performing step 3 of 5
creating bidirectional link between nodes 27 (src) and tensor([33]) (dst)
created 1 of 1 links requested
performing step 4 of 5
creating bidirectional link between nodes 36 (src) and tensor([62]) (dst)
created 1 of 1 links requested
performing step 5 of 5
all FoF nodes are already direcctly connected to node 24.
no FoF link possible for src dst nodes 24 and 2.
created 0 of 1 links requested
